In [1]:
%cd '/Users/maryamalki/Desktop/PYRICE_2022'

/Users/maryamalki/Desktop/PyRICE_2022


In [2]:
from model.data.regions import (
    Africa_countries,
    Latin_America_countries,
    Middle_East_countries,
    Non_OECD_Asia_countries,
    Non_Russia_Eurasia_countries,
    OECD_Europe_countries,
    OHI_countries,
    Region,
    combined_regions,
)

In [3]:
import pandas as pd
path = '/Users/maryamalki/Desktop/PYRICE_2022/model/inputdata/mpd2020.xlsx'

In [4]:
df_gdp_pc = pd.read_excel(path, sheet_name="GDP pc", index_col="GDP pc 2011 prices")

In [5]:
df_gdp_pc = df_gdp_pc.loc[1:]
df_gdp_pc.index.name = None
df_gdp_pc = df_gdp_pc.loc[1990:]

In [6]:
df_pop = pd.read_excel(path, sheet_name="Population", index_col="Population")
df_pop = df_pop.loc[1:]
df_pop.index.name = None

In [7]:
df_pop = df_pop.loc[1990:]

In [8]:
df_gdp = df_gdp_pc * df_pop * 1000  # Population in thousands

In [9]:
# Reconcile countries' names between datasets
rename = {
        "Bolivia (Plurinational State of)": "Bolivia",
        "Cabo Verde": "Cape Verde",
        "China, Hong Kong SAR": "Hong Kong",
        "Côte d'Ivoire": "Cote d'Ivoire",
        "D.P.R. of Korea": "North Korea",
        "D.R. of the Congo": "Democratic Republic of Congo",
        "Former USSR": "USSR",
        "Former Yugoslavia": "Yugoslavia",
        "Iran (Islamic Republic of)": "Iran",
        "Lao People's DR": "Laos",
        "Republic of Korea": "South Korea",
        "Republic of Moldova": "Moldova",
        "Russian Federation": "Russia",
        "Saint Lucia": "St. Lucia",
        "State of Palestine": "Palestinian Territory",
        "Sudan (Former)": "Sudan",
        "Swaziland": "Eswatini",
        "Syrian Arab Republic": "Syria",
        "TFYR of Macedonia": "North Macedonia",
        "Taiwan, Province of China": "Taiwan",
        "U.R. of Tanzania: Mainland": "Tanzania",
        "United States": "US",
        "Venezuela (Bolivarian Republic of)": "Venezuela",
        "Viet Nam": "Vietnam",
    }

df_gdp.rename(rename, axis=1, inplace=True)

combined_lower_to_capitalized = {country.lower(): country for country in combined_regions}

# Rename country GDP columns to all lowercase
countries = set(country.lower() for country in df_gdp.columns)

df_gdp.rename({k: v for k, v in zip(df_gdp.columns, map(lambda x: x.lower(), df_gdp.columns))}, axis=1, inplace=True)

combined_lower = list(map(lambda x: x.lower(), combined_regions))

excluded_countries_from_ours = countries - set(combined_lower)
excluded_countries_from_theirs = set(combined_lower) - countries

df_gdp.drop(labels=excluded_countries_from_ours, axis=1, inplace=True)
    
# Rename country GDP columns based on lower combined
df_gdp.rename(combined_lower_to_capitalized, axis=1, inplace=True)

# Single Country Regions
US_mpd_gdp = df_gdp[Region.US.value]
Russia_mpd_gdp = df_gdp[Region.Russia.value]
China_mpd_gdp = df_gdp[Region.China.value]
Japan_mpd_gdp = df_gdp[Region.Japan.value]
India_mpd_gdp = df_gdp[Region.India.value]

# Simple Grouped Regions
Africa_cleaned = list(set(Africa_countries) - set(combined_lower_to_capitalized[c] for c in excluded_countries_from_theirs))
Middle_East_cleaned = list(set(Middle_East_countries) - set(combined_lower_to_capitalized[c] for c in excluded_countries_from_theirs))
Non_Russia_Eurasia_cleaned = list(set(Non_Russia_Eurasia_countries) - set(combined_lower_to_capitalized[c] for c in excluded_countries_from_theirs))
Latin_America_cleaned = list(set(Latin_America_countries) - set(combined_lower_to_capitalized[c] for c in excluded_countries_from_theirs))
Non_OECD_Asia_cleaned = list(set(Non_OECD_Asia_countries) - set(combined_lower_to_capitalized[c] for c in excluded_countries_from_theirs))
OECD_Europe_cleaned = list(set(OECD_Europe_countries) - set(combined_lower_to_capitalized[c] for c in excluded_countries_from_theirs))
OHI_cleaned = list(set(OHI_countries) - set(combined_lower_to_capitalized[c] for c in excluded_countries_from_theirs))

Africa_mpd_gdp = df_gdp.loc[:, Africa_cleaned].sum(axis=1)
Middle_East_mpd_gdp = df_gdp.loc[:, Middle_East_cleaned].sum(axis=1)
Non_Russia_Eurasia_mpd_gdp = df_gdp.loc[:, Non_Russia_Eurasia_cleaned].sum(axis=1)
Latin_America_mpd_gdp = df_gdp.loc[:, Latin_America_cleaned].sum(axis=1)
Non_OECD_Asia_mpd_gdp = df_gdp.loc[:, Non_OECD_Asia_cleaned].sum(axis=1)
OECD_Europe_mpd_gdp = df_gdp.loc[:, OECD_Europe_cleaned].sum(axis=1)
OHI_mpd_gdp = df_gdp.loc[:, OHI_cleaned].sum(axis=1)

# Use common names
Africa_mpd_gdp.name = Region.Africa.value
Middle_East_mpd_gdp.name = Region.Middle_East.value
Non_Russia_Eurasia_mpd_gdp.name = Region.Non_Russia_Eurasia.value
Latin_America_mpd_gdp.name = Region.Latin_America.value
Non_OECD_Asia_mpd_gdp.name = Region.Non_OECD_Asia.value
OECD_Europe_mpd_gdp.name = Region.OECD_Europe.value
OHI_mpd_gdp.name = Region.OHI.value

mpd_gdp = pd.DataFrame(data=
{
            Region.US.value: US_mpd_gdp,
            Region.OECD_Europe.value: OECD_Europe_mpd_gdp,
            Region.Japan.value: Japan_mpd_gdp,
            Region.Russia.value: Russia_mpd_gdp,
            Region.Non_Russia_Eurasia.value: Non_Russia_Eurasia_mpd_gdp,
            Region.China.value: China_mpd_gdp,
            Region.India.value: India_mpd_gdp,
            Region.Middle_East.value: Middle_East_mpd_gdp,
            Region.Africa.value: Africa_mpd_gdp,
            Region.Latin_America.value: Latin_America_mpd_gdp,
            Region.OHI.value: OHI_mpd_gdp,
            Region.Non_OECD_Asia.value: Non_OECD_Asia_mpd_gdp,
        }
    )

# Adjust prices from 2011$ price to 2005$ price
price_adjust = 1.15 
mpd_gdp = mpd_gdp/price_adjust

In [10]:
# Display with Scientific Notation
pd.set_option ('display.float_format', lambda x: "%.12E" % x)


mpd_gdp

,US,OECD-Europe,Japan,Russia,Non-Russia Eurasia,China,India,Middle East,Africa,Latin America,OHI,Other non-OECD Asia
1990,8.043806699050E+12,9.442040977518E+12,3.217236141436E+12,1.595534956522E+12,1.418131183167E+12,2.943584060870E+12,1.522602608696E+12,1.313855138970E+12,7.873253838614E+11,3.084155188143E+12,2.188718882991E+12,1.945532105332E+12
1991,8.037825315304E+12,9.597575276289E+12,3.325824777241E+12,1.545640458031E+12,1.294970575076E+12,3.051259016724E+12,1.532531723515E+12,1.368147574173E+12,7.979150028986E+11,3.221947610258E+12,2.254571058372E+12,2.078062469474E+12
1992,8.323607031973E+12,9.770833746081E+12,3.352800167340E+12,1.352185411150E+12,1.130349441307E+12,3.265250031501E+12,1.602052622840E+12,1.500396360161E+12,8.003380532655E+11,3.351481768906E+12,2.345975857587E+12,2.202466775660E+12
1993,8.552122981825E+12,9.860107169102E+12,3.357458526073E+12,1.260429846690E+12,1.022443598045E+12,3.587973542715E+12,1.679887373907E+12,1.604487053218E+12,8.159447632115E+11,3.501359781091E+12,2.465008453936E+12,2.346552930984E+12
1994,8.897410511323E+12,1.016377858777E+13,3.390153925065E+12,1.126594216169E+12,9.136111504383E+11,3.859785587054E+12,1.773499751444E+12,1.690053643899E+12,8.395912877565E+11,3.691822673550E+12,2.624433314669E+12,2.510670381922E+12
1995,9.139359482940E+12,1.053478356839E+13,3.480898629930E+12,1.104820918748E+12,8.841793514998E+11,4.182608721131E+12,1.887861722253E+12,1.780157861246E+12,8.817330966085E+11,3.768231056025E+12,2.762980966737E+12,2.700410981082E+12
1996,9.486232147613E+12,1.085191531295E+13,3.586650459586E+12,1.091060357832E+12,8.847154819465E+11,4.458773596021E+12,2.020637377598E+12,1.928344342090E+12,9.400327424307E+11,3.926184374162E+12,2.891584781226E+12,2.885865445260E+12
1997,9.911962648464E+12,1.127381994146E+13,3.623078926414E+12,1.130209469417E+12,9.072243794341E+11,4.601164252215E+12,2.089664468236E+12,2.042189398809E+12,9.897417195195E+11,4.162942795238E+12,3.036937870971E+12,2.993316020022E+12
1998,1.035306849666E+13,1.169908364767E+13,3.580005992566E+12,1.095171590606E+12,9.258152508474E+11,4.642350449208E+12,2.209994955473E+12,2.189749495604E+12,1.030538961494E+12,4.291011883245E+12,3.054384424751E+12,2.833202679935E+12
1999,1.083810367914E+13,1.208503969515E+13,3.568718834016E+12,1.190456485326E+12,9.408002076258E+11,4.850872058767E+12,2.331583640770E+12,2.291356253236E+12,1.070028300554E+12,4.331762604825E+12,3.256071500049E+12,2.938105419289E+12


In [11]:
mpd_gdp_2005 = mpd_gdp.loc[2005]

In [12]:
mpd_gdp_2005


US                    1.278585385724E+13
OECD-Europe           1.451609248267E+13
Japan                 3.875420816322E+12
Russia                2.018159267690E+12
Non-Russia Eurasia    1.473293007615E+12
China                 7.480817344867E+12
India                 3.225540075848E+12
Middle East           3.721438138126E+12
Africa                1.601353491188E+12
Latin America         5.393047850726E+12
OHI                   4.144771220258E+12
Other non-OECD Asia   4.097925313170E+12
Name: 2005, dtype: object

In [13]:
# Reconcile countries' names between datasets
rename = {
        "Bolivia (Plurinational State of)": "Bolivia",
        "Cabo Verde": "Cape Verde",
        "China, Hong Kong SAR": "Hong Kong",
        "Côte d'Ivoire": "Cote d'Ivoire",
        "D.P.R. of Korea": "North Korea",
        "D.R. of the Congo": "Democratic Republic of Congo",
        "Former USSR": "USSR",
        "Former Yugoslavia": "Yugoslavia",
        "Iran (Islamic Republic of)": "Iran",
        "Lao People's DR": "Laos",
        "Republic of Korea": "South Korea",
        "Republic of Moldova": "Moldova",
        "Russian Federation": "Russia",
        "Saint Lucia": "St. Lucia",
        "State of Palestine": "Palestinian Territory",
        "Sudan (Former)": "Sudan",
        "Swaziland": "Eswatini",
        "Syrian Arab Republic": "Syria",
        "TFYR of Macedonia": "North Macedonia",
        "Taiwan, Province of China": "Taiwan",
        "U.R. of Tanzania: Mainland": "Tanzania",
        "United States": "US",
        "Venezuela (Bolivarian Republic of)": "Venezuela",
        "Viet Nam": "Vietnam",
    }

df_pop.rename(rename, axis=1, inplace=True)

combined_lower_to_capitalized = {country.lower(): country for country in combined_regions}

# Rename country GDP columns to all lowercase
countries = set(country.lower() for country in df_pop.columns)

df_pop.rename({k: v for k, v in zip(df_pop.columns, map(lambda x: x.lower(), df_pop.columns))}, axis=1, inplace=True)

combined_lower = list(map(lambda x: x.lower(), combined_regions))

excluded_countries_from_ours = countries - set(combined_lower)
excluded_countries_from_theirs = set(combined_lower) - countries

df_pop.drop(labels=excluded_countries_from_ours, axis=1, inplace=True)
    
# Rename country GDP columns based on lower combined
df_pop.rename(combined_lower_to_capitalized, axis=1, inplace=True)

# Single Country Regions
US_mpd_pop = df_pop[Region.US.value]
Russia_mpd_pop = df_pop[Region.Russia.value]
China_mpd_pop = df_pop[Region.China.value]
Japan_mpd_pop = df_pop[Region.Japan.value]
India_mpd_pop = df_pop[Region.India.value]

# Simple Grouped Regions
Africa_cleaned = list(set(Africa_countries) - set(combined_lower_to_capitalized[c] for c in excluded_countries_from_theirs))
Middle_East_cleaned = list(set(Middle_East_countries) - set(combined_lower_to_capitalized[c] for c in excluded_countries_from_theirs))
Non_Russia_Eurasia_cleaned = list(set(Non_Russia_Eurasia_countries) - set(combined_lower_to_capitalized[c] for c in excluded_countries_from_theirs))
Latin_America_cleaned = list(set(Latin_America_countries) - set(combined_lower_to_capitalized[c] for c in excluded_countries_from_theirs))
Non_OECD_Asia_cleaned = list(set(Non_OECD_Asia_countries) - set(combined_lower_to_capitalized[c] for c in excluded_countries_from_theirs))
OECD_Europe_cleaned = list(set(OECD_Europe_countries) - set(combined_lower_to_capitalized[c] for c in excluded_countries_from_theirs))
OHI_cleaned = list(set(OHI_countries) - set(combined_lower_to_capitalized[c] for c in excluded_countries_from_theirs))

Africa_mpd_pop = df_pop.loc[:, Africa_cleaned].sum(axis=1)
Middle_East_mpd_pop = df_pop.loc[:, Middle_East_cleaned].sum(axis=1)
Non_Russia_Eurasia_mpd_pop = df_pop.loc[:, Non_Russia_Eurasia_cleaned].sum(axis=1)
Latin_America_mpd_pop = df_pop.loc[:, Latin_America_cleaned].sum(axis=1)
Non_OECD_Asia_mpd_pop = df_pop.loc[:, Non_OECD_Asia_cleaned].sum(axis=1)
OECD_Europe_mpd_pop = df_pop.loc[:, OECD_Europe_cleaned].sum(axis=1)
OHI_mpd_pop = df_pop.loc[:, OHI_cleaned].sum(axis=1)

# Use common names
Africa_mpd_pop.name = Region.Africa.value
Middle_East_mpd_pop.name = Region.Middle_East.value
Non_Russia_Eurasia_mpd_pop.name = Region.Non_Russia_Eurasia.value
Latin_America_mpd_pop.name = Region.Latin_America.value
Non_OECD_Asia_mpd_pop.name = Region.Non_OECD_Asia.value
OECD_Europe_mpd_pop.name = Region.OECD_Europe.value
OHI_mpd_pop.name = Region.OHI.value

mpd_pop = pd.DataFrame(data=
{
            Region.US.value: US_mpd_pop,
            Region.OECD_Europe.value: OECD_Europe_mpd_pop,
            Region.Japan.value: Japan_mpd_pop,
            Region.Russia.value: Russia_mpd_pop,
            Region.Non_Russia_Eurasia.value: Non_Russia_Eurasia_mpd_pop,
            Region.China.value: China_mpd_pop,
            Region.India.value: India_mpd_pop,
            Region.Middle_East.value: Middle_East_mpd_pop,
            Region.Africa.value: Africa_mpd_pop,
            Region.Latin_America.value: Latin_America_mpd_pop,
            Region.OHI.value: OHI_mpd_pop,
            Region.Non_OECD_Asia.value: Non_OECD_Asia_mpd_pop,
        }
    )

In [14]:
# Display with Scientific Notation
pd.set_option ('display.float_format', lambda x: "%.6F" % x)

mpd_pop

,US,OECD-Europe,Japan,Russia,Non-Russia Eurasia,China,India,Middle East,Africa,Latin America,OHI,Other non-OECD Asia
1990,250131.894000,508285.373000,123537.399000,147973,187674.880000,1135185,839000,254049.584000,499494.412114,440329.389000,119936.659000,735392.828000
1991,253496.869554,511376.736044,124027.793575,147973,188473.599685,1149978.853481,854581.181462,260534.885704,514787.873449,448333.194157,121337.580255,750396.141843
1992,257037.358320,514389.963061,124467.147081,148272.338503,188762.365410,1163362.822938,870986.424116,266741.654699,529615.084402,456313.552097,122713.739042,765671.550778
1993,260448.665170,517178.190988,124870.471599,148072.779501,188866.680111,1176726.934874,887633.635364,272835.747579,543799.180289,464166.742728,124095.668943,781537.195349
1994,263662.438991,519583.509811,125216.750217,148172.559002,188713.302013,1189961.972921,904486.118414,278807.646975,556693.714074,472004.912734,125478.905534,797587.055356
1995,266821.440337,521842.411747,125537.007898,147973.000000,188456.354222,1202581.427803,921508.862166,284598.209114,571261.908771,479730.485520,126943.586044,813378.526316
1996,269943.685875,524036.550621,125812.229343,147973.000000,188425.501514,1215171.096403,938666.572232,290193.682887,586304.140306,487374.110887,128364.852735,829149.779288
1997,273202.960301,526146.639167,126112.470920,147474.102495,188338.355698,1227452.973420,955884.401574,296060.024062,601093.231881,494836.212833,129678.260068,844457.300495
1998,276416.680012,528181.292890,126450.743096,147174.763992,188221.849639,1238722.116843,973088.413061,301866.811734,616286.269217,502302.836834,130835.724994,859550.877948
1999,279609.241661,530347.016298,126688.934746,146576.086986,188226.580669,1248899.096586,990242.991988,307738.869024,632371.827081,509514.243494,131952.239052,874836.196173


In [15]:
mpd_pop.loc[2005]/1000

US                     296.119275
OECD-Europe            546.853193
Japan                  127.854873
Russia                 142.884245
Non-Russia Eurasia     187.182137
China                 1298.245037
India                 1092.067921
Middle East            346.876438
Africa                 739.608110
Latin America          551.280180
OHI                    138.551562
Other non-OECD Asia    965.596627
Name: 2005, dtype: object